# Векторизация, Numba

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. "Оптимизация выполнения кода, векторизация, Numba"
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html
* https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types
* https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html
* https://numba.pydata.org/numba-doc/latest/user/vectorize.html


In [11]:
import numpy as np
import numba

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`. Ускорьте вычисления при помощи numba

In [ ]:
A = np.random.randint(0, 1000, size=(1000000,))

In [ ]:
def f1(A):
    acc, cnt = 0, 0
    for x in A:
        acc += x + 100
        cnt += 1
    return acc / cnt

In [ ]:
%%time
f1(A)

CPU times: user 341 ms, sys: 2.17 ms, total: 343 ms
Wall time: 349 ms


599.553201

In [ ]:
@numba.njit
def smart_f(A):
    acc, cnt = 0, 0
    for x in A:
        acc += x + 100
        cnt += 1
    return acc / cnt

In [ ]:
%%time
smart_f(A)

CPU times: user 450 ms, sys: 120 ms, total: 570 ms
Wall time: 505 ms


599.553201

In [ ]:
%%time
smart_f(A)

CPU times: user 992 µs, sys: 0 ns, total: 992 µs
Wall time: 1 ms


599.553201

2. Напишите функцию, которая возвращает сумму всех чисел от 0 до x-1. Создайте массив, заполненный случайными целыми неотрицательными числами и примените функцию к каждому элементу массива.

In [ ]:
def smart_sum(x):
  return sum(range(x))

In [ ]:
smart_sum(12312)

75786516

In [ ]:
A = np.random.randint(0,100,size=1_000_000)

In [ ]:
%%time
r = np.array([smart_sum(a) for a in A])
r[:5]

CPU times: user 771 ms, sys: 18.2 ms, total: 790 ms
Wall time: 797 ms


array([1830,  276,  528, 1953,  903])

In [ ]:
smart_sum(A)

TypeError: ignored

In [ ]:
@np.vectorize
def smart_sum_npv(x):
  return sum(range(x))

In [ ]:
%%time
smart_sum_npv(A)

CPU times: user 732 ms, sys: 31.4 ms, total: 763 ms
Wall time: 766 ms


array([1830,  276,  528, ..., 3160, 1653,    0])

In [ ]:
@numba.vectorize
def smart_sum_nbv(x):
  return sum(range(x))

In [ ]:
%%time
smart_sum_nbv(A)

CPU times: user 150 ms, sys: 2.84 ms, total: 153 ms
Wall time: 155 ms


array([1830,  276,  528, ..., 3160, 1653,    0])

3. Приведите все слова из столбца key к верхнему регистру

In [ ]:
import pandas as pd
import string
import numpy as np

def create_df(allow_nan=False, N=2_000_000):
    df = pd.DataFrame(np.random.randint(0, 10, (N, 4)), columns=[f"col{i}" for i in range(4)])
    names = ["Apple",  "Banana",  "Apricot",  "Atemoya",  "Avocados",  "Blueberry",  "Blackcurrant",  "Ackee",  "Cranberry",  "Cantaloupe",  "Cherry",  "Black sapote/Chocolate pudding fruit",  "Dragonrfruit",  "Dates",  "Cherimoya",  "Buddha’s hand fruit",  "Finger Lime",  "Fig",  "Coconut",]
    if allow_nan:
        names.append(None)
    df["key"] = np.random.choice(names, N, replace=True)
    return df

In [ ]:
%%time
df = create_df(allow_nan=False, N=2_000_000)

CPU times: user 462 ms, sys: 958 ms, total: 1.42 s
Wall time: 1.42 s


In [ ]:
df

,col0,col1,col2,col3,key
0,3,6,0,3,Buddha’s hand fruit
1,0,9,0,7,Cranberry
2,9,9,2,5,Apricot
3,4,5,3,9,Buddha’s hand fruit
4,9,5,6,4,Blueberry
...,...,...,...,...,...
1999995,7,1,3,6,Apple
1999996,1,5,7,3,Coconut
1999997,2,9,6,4,Cantaloupe
1999998,1,5,0,7,Apricot


In [ ]:
%%time
df['key'].map(str.upper)

CPU times: user 343 ms, sys: 97.3 ms, total: 440 ms
Wall time: 661 ms


0          BUDDHA’S HAND FRUIT
1                    CRANBERRY
2                      APRICOT
3          BUDDHA’S HAND FRUIT
4                    BLUEBERRY
                  ...         
1999995                  APPLE
1999996                COCONUT
1999997             CANTALOUPE
1999998                APRICOT
1999999              CHERIMOYA
Name: key, Length: 2000000, dtype: object

In [ ]:
%%time
df['key'].str.upper()

CPU times: user 943 ms, sys: 104 ms, total: 1.05 s
Wall time: 1.1 s


0             CRANBERRY
1           FINGER LIME
2          DRAGONRFRUIT
3                 DATES
4            CANTALOUPE
               ...     
1999995         ATEMOYA
1999996         COCONUT
1999997         COCONUT
1999998          BANANA
1999999             FIG
Name: key, Length: 2000000, dtype: object

In [ ]:
def smart_d(row):
  

In [ ]:
np.where(
    df['col3'] % 2 == 0,
    print('четн'),
    print('нечет')
)

четн
нечет


array([None, None, None, ..., None, None, None], dtype=object)

4\. Для каждой строки рассчитайте разность между значениями col0 и col1, если в столбце col3 стоит четное число, и разность между col0 и col2 в противном случае.

In [ ]:
df = create_df(allow_nan=False, N=500_000).select_dtypes('number')

## Лабораторная работа 2

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy` и `pandas`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy` или структур `pandas` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

In [1]:
import pandas as pd

In [2]:
recipes = pd.read_csv('/content/recipes_sample.csv')
reviews = pd.read_csv('/content/reviews_sample.csv')

In [3]:
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [4]:
reviews.head()

,Unnamed: 0,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


## Numba

В файле `rating_predictions.json` хранятся данные о рейтингах рецептов и прогнозных значениях рейтингов для этого рецепта, полученных при помощи модели машинного обучения. 

Напишите несколько версий функции (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от прогнозного значения рейтинга для данного рецепта. 


Замечание 1: в формуле MAPE под $A_t$ понимается рейтинг из отзыва $t$, под $F_t$ - прогнозное значения рейтинга отзыва $t$.

Замечание 2: в результате работы функций должно получиться одно число - MAPE для всего набора данных.

№1\.1 Создайте два списка `A_list` и `F_list` на основе файла `rating_predictions.json`. Напишите функцию `mape_lists` без использования векторизованных операций и методов массивов `numpy` и без использования `numba` (проитерируйтесь по спискам и вычислите суммарное значение MAPE для всех элементов, а потом усредните результат).

Измерьте время выполнения данной функции на входных данных `A_list` и `F_list`. Временем, затрачиваемым на создание списков, можно пренебречь.
    

In [6]:
df = pd.read_json('rating_predictions.json')
print(df)

        rating  prediction
0            5    4.944444
1            5    4.437500
2            5    4.727273
3            5    4.354545
4            5    4.888889
...        ...         ...
119886       5    4.903226
119887       5    4.333333
119888       5    5.000000
119889       5    4.142857
119890       5    5.000000

[119891 rows x 2 columns]


In [7]:
A_list, F_list = list(df['rating']),  list(df['prediction'])

In [14]:
def mape_lists(A, F):
  s = 0
  n = len(A)
  for i in range(n):
    s += abs((A[i]-F[i])/A[i])
  return s/n * 100

In [15]:
%%time
mape_lists(A_list, F_list)

CPU times: user 54.6 ms, sys: 604 µs, total: 55.2 ms
Wall time: 119 ms


13.325265503992636

№1\.2. Создайте массивы `numpy` `A_array` и `F_array` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numpy` с использованием векторизованных операций и методов массивов `numpy`.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`. Временем, затрачиваемым на создание массивов, можно пренебречь.

In [12]:
A_list_arr, F_list_arr = np.array(A_list), np.array(F_list)

In [17]:
def mape_numpy(A, F):
  return abs((A_list_arr-F_list_arr)/A_list_arr).sum()/len(A_list_arr)*100

In [18]:
%%time
mape_numpy(A_list_arr, F_list_arr)

CPU times: user 605 µs, sys: 984 µs, total: 1.59 ms
Wall time: 1.62 ms


13.32526550399145

№1\.3. Создайте объекты `numba.typed.List` `A_typed` и `F_typed` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numba` без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`. 

Измерьте время выполнения данной функции на входных данных `A_typed` и `F_typed`. Временем, затрачиваемым на создание объектов `numba.typed.List`, можно пренебречь.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`.

In [19]:
from numba.typed import List

In [24]:
@numba.njit
def mape_numba(A, F):
  s = 0
  n = len(A)
  for i in range(n):
    s += abs((A[i]-F[i])/A[i])
  return s/n * 100

In [25]:
A_list_List, F_list_List = List(A_list), List(F_list)

In [27]:
%%time
mape_numba(A_list_List, F_list_List)

CPU times: user 2.07 ms, sys: 0 ns, total: 2.07 ms
Wall time: 2.08 ms


13.325265503992636

#### Вывод:

In [32]:
%%timeit
mape_lists(A_list, F_list)

36.8 ms ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%%timeit
mape_numpy(A_list_arr, F_list_arr)

859 µs ± 434 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [34]:
%%timeit
mape_numba(A_list_List, F_list_List)

1.81 ms ± 171 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


* Без векторизованных операций и numba вычисления занимают 36.8 ms
* Numpy уменьшает время в $\sim 45$ раз
* Numba ускоряет в $\sim 21$ раз

## Векторизация

Сайт-агрегатор устроил акцию: он дарит купоны на посещение ресторана тем пользователям, оставившим отзывы, идентификатор которых является _красивым числом_. Натуральное число называется _красивым_, если первая цифра числа совпадает с последней цифрой числа. 



In [39]:
ids = reviews["user_id"].unique()
ids[:5]

array([     21752,     431813,     400708, 2001852463,      95810])

№2\.1 Напишите функцию `is_pretty`, которая для каждого идентификатора пользователя из файла определяет, получит ли он подарок. Запрещается преобразовывать идентификатор пользователя к строке. Подтвердите корректность реализации, продемонстрировав примеры.

In [40]:
def is_pretty(n: int) -> bool:
    last = n % 10
    while n > 9:
        n //= 10
    return True if n==last else False

In [41]:
is_pretty(121)

True

In [42]:
is_pretty(221)

False

№2\.2 Посчитайте с помощью функции `is_pretty` количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.

In [45]:
%%timeit
c = 0
for i in ids:
  if is_pretty(i):
    c += 1
c

135 ms ± 25.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
%%timeit
sum(list(map(is_pretty, ids)))

128 ms ± 8.81 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


№2\.3. При помощи декоратора `numpy.vectorize` создайте векторизованную версию функции `is_pretty`. Корректно использовав эту векторизованную функцию, посчитайте количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [49]:
@np.vectorize
def is_pretty_numpy_vec(n: int) -> bool:
    last = n % 10
    while n > 9:
        n //= 10
    return True if n==last else False

In [57]:
%%timeit
is_pretty_numpy_vec(ids).sum()

34.1 ms ± 804 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


№2\.4. При помощи декоратора `numba.vectorize` создайте векторизованную версию функции `is_pretty`. Корректно использовав эту векторизованную функцию, посчитайте количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [58]:
@numba.vectorize
def is_pretty_numba_vec(n: int) -> bool:
    last = n % 10
    while n > 9:
        n //= 10
    return True if n==last else False

In [60]:
%%timeit
is_pretty_numba_vec(ids).sum()

846 µs ± 63.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### Вывод:

In [63]:
%%timeit
c = 0
for i in ids:
  if is_pretty(i):
    c += 1
c

155 ms ± 81.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [64]:
%%timeit
is_pretty_numpy_vec(ids).sum()

38.4 ms ± 9.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [65]:
%%timeit
is_pretty_numba_vec(ids).sum()

859 µs ± 80.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


* Время без векторизации $\sim 155$ ms
* Векторизаций numpy ускоряет процесс в $\sim 4$ раза
* Векторизаций numba ускоряет процесс в $\sim 180$ раз